In [1]:
from plyfile import PlyData

In [2]:
data = PlyData.read('clean_pinky.ply')

In [3]:
print(data)

ply
format binary_little_endian 1.0
element vertex 134694
property float x
property float y
property float z
property float nx
property float ny
property float nz
property float f_dc_0
property float f_dc_1
property float f_dc_2
property float f_rest_0
property float f_rest_1
property float f_rest_2
property float f_rest_3
property float f_rest_4
property float f_rest_5
property float f_rest_6
property float f_rest_7
property float f_rest_8
property float f_rest_9
property float f_rest_10
property float f_rest_11
property float f_rest_12
property float f_rest_13
property float f_rest_14
property float f_rest_15
property float f_rest_16
property float f_rest_17
property float f_rest_18
property float f_rest_19
property float f_rest_20
property float f_rest_21
property float f_rest_22
property float f_rest_23
property float f_rest_24
property float f_rest_25
property float f_rest_26
property float f_rest_27
property float f_rest_28
property float f_rest_29
property float f_rest_30
proper

In [17]:
import open3d as o3d

# Load the PLY file
ply_file = "C:/Users/nguye/Downloads/image2/test2.ply"
pcd = o3d.io.read_point_cloud(ply_file)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])


In [ ]:
import open3d as o3d

# Load the PLY file
ply_file = "C:/Users/nguye/Downloads/image2/test2.ply"
pcd = o3d.io.read_point_cloud(ply_file)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])


In [5]:
import open3d as o3d
import numpy as np

def create_infinite_axes():
    """
    Creates a set of long lines along the X, Y, and Z axes to simulate infinite axes.
    """
    # Define start and end points for each axis (very long lines)
    length = 100  # Large number to simulate infinite axes
    points = np.array([
        [-length, 0, 0], [length, 0, 0],  # X-axis (red)
        [0, -length, 0], [0, length, 0],  # Y-axis (green)
        [0, 0, -length], [0, 0, length]   # Z-axis (blue)
    ])
    
    # Define line connections (pairs of points)
    lines = [[0, 1], [2, 3], [4, 5]]
    
    # Define colors: Red for X, Green for Y, Blue for Z
    colors = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]

    # Create LineSet
    axis_lines = o3d.geometry.LineSet()
    axis_lines.points = o3d.utility.Vector3dVector(points)
    axis_lines.lines = o3d.utility.Vector2iVector(lines)
    axis_lines.colors = o3d.utility.Vector3dVector(colors)

    return axis_lines

def plot_point_clouds_with_infinite_axes(ply_files):
    """
    Plots multiple PLY point clouds along with infinite coordinate axes.

    :param ply_files: List of file paths to PLY point cloud files.
    """
    geometries = [create_infinite_axes()]  # Add the infinite axis

    for file in ply_files:
        pcd = o3d.io.read_point_cloud(file)
        if not pcd.is_empty():
            geometries.append(pcd)
        else:
            print(f"Warning: {file} is empty or could not be loaded.")

    if geometries:
        o3d.visualization.draw_geometries(geometries)
    else:
        print("No valid point clouds to display.")

# Example usage
ply_list = [
    # "C:/Users/nguye/Downloads/images/test.ply",
    "C:/Users/nguye/Downloads/image2/test2.ply",
    # "corner2_short.ply"
    # "corner2_voxel.ply",
    # "corner1_voxel.ply",
]
plot_point_clouds_with_infinite_axes(ply_list)


In [21]:
def draw_registration_result(source, target, transformation):
    source_temp = source.clone()
    target_temp = target.clone()

    source_temp.transform(transformation)

    # This is patched version for tutorial rendering.
    # Use `draw` function for you application.
    o3d.visualization.draw_geometries(
        [source_temp.to_legacy(),
         target_temp.to_legacy()],
        zoom=0.4459,
        front=[0.9288, -0.2951, -0.2242],
        lookat=[1.6784, 2.0612, 1.4451],
        up=[-0.3402, -0.9189, -0.1996])

In [22]:
demo_icp_pcds = o3d.data.DemoICPPointClouds()
source = o3d.t.io.read_point_cloud(demo_icp_pcds.paths[0])
target = o3d.t.io.read_point_cloud(demo_icp_pcds.paths[1])

# For Colored-ICP `colors` attribute must be of the same dtype as `positions` and `normals` attribute.
source.point["colors"] = source.point["colors"].to(
    o3d.core.Dtype.Float32) / 255.0
target.point["colors"] = target.point["colors"].to(
    o3d.core.Dtype.Float32) / 255.0

# Initial guess transform between the two point-cloud.
# ICP algortihm requires a good initial allignment to converge efficiently.
trans_init = np.asarray([[0.862, 0.011, -0.507, 0.5],
                         [-0.139, 0.967, -0.215, 0.7],
                         [0.487, 0.255, 0.835, -1.4], [0.0, 0.0, 0.0, 1.0]])

draw_registration_result(source, target, trans_init)

In [17]:
import open3d as o3d
import numpy as np
import plyfile

def read_splat_ply(file_path, num_vertices=None):
    """
    Reads a splat PLY file and extracts positions, colors, scales, and opacity.
    Supports subsampling to a given number of vertices.
    """
    ply_data = plyfile.PlyData.read(file_path)
    vertex_data = ply_data["vertex"]

    # Extract positions, colors, scales, and opacity
    positions = np.vstack([vertex_data["x"], vertex_data["y"], vertex_data["z"]]).T
    colors = np.vstack([vertex_data["f_dc_0"], vertex_data["f_dc_1"], vertex_data["f_dc_2"]]).T
    colors = np.clip(colors, 0, 1)  # Normalize to range [0,1]
    scales = np.vstack([vertex_data["scale_0"], vertex_data["scale_1"], vertex_data["scale_2"]]).T
    opacity = np.array(vertex_data["opacity"])

    # Subsampling
    if num_vertices is not None and num_vertices < len(positions):
        indices = np.random.choice(len(positions), num_vertices, replace=False)
        positions, colors, scales, opacity = positions[indices], colors[indices], scales[indices], opacity[indices]

    return positions, colors, scales, opacity

def create_splats(positions, colors, scales, opacity, sphere_resolution=5):
    """
    Creates Gaussian splats by rendering each point as a small sphere.
    """
    splats = []
    for i in range(len(positions)):
        sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.5, resolution=sphere_resolution)
        
        # Translate to correct position
        sphere.translate(positions[i])
        
        # Scale according to splat scale
        # sphere.scale(scales[i][0], center=sphere.get_center())  # Approximate scaling
        
        # Color with opacity
        sphere.paint_uniform_color([colors[i][0], colors[i][1], colors[i][2]])

        splats.append(sphere)

    return splats

def visualize_gaussian_splats(ply_file, num_vertices=None):
    """
    Reads the PLY file, subsamples it (if specified), and visualizes it as Gaussian splats.
    """
    positions, colors, scales, opacity = read_splat_ply(ply_file, num_vertices)
    splats = create_splats(positions, colors, scales, opacity)
    tes = "C:/Users/nguye/Downloads/image2/test2.ply"
    pcd = o3d.io.read_point_cloud(tes)

    # Visualize the point cloud
    o3d.visualization.draw_geometries([pcd] + splats)

# Example usage:
ply_file_path = "corner2_short.ply"
visualize_gaussian_splats(ply_file_path, num_vertices=2000)  # Subsamples to 1000 points


In [7]:
import open3d as o3d
import numpy as np
import plyfile
import copy

def read_splat_ply(file_path, num_vertices=None):
    """
    Reads a splat PLY file and extracts positions, colors, scales, and opacity.
    Supports subsampling to a given number of vertices.
    """
    ply_data = plyfile.PlyData.read(file_path)
    vertex_data = ply_data["vertex"]

    # Extract positions, colors, scales, and opacity
    positions = np.vstack([vertex_data["x"], vertex_data["y"], vertex_data["z"]]).T
    colors = np.vstack([vertex_data["f_dc_0"], vertex_data["f_dc_1"], vertex_data["f_dc_2"]]).T
    colors = np.clip(colors, 0, 1)  # Normalize to range [0,1]
    scales = np.vstack([vertex_data["scale_0"], vertex_data["scale_1"], vertex_data["scale_2"]]).T
    opacity = np.array(vertex_data["opacity"])
    
    # Extract rotation quaternion (assumes stored as [w, x, y, z])
    rotations = np.vstack([vertex_data["rot_0"], vertex_data["rot_1"], vertex_data["rot_2"], vertex_data["rot_3"]]).T

    # Subsampling
    if num_vertices is not None and num_vertices < len(positions):
        indices = np.random.choice(len(positions), num_vertices, replace=False)
        positions, colors, scales, opacity, rotations = (
            positions[indices], colors[indices], scales[indices], opacity[indices], rotations[indices]
        )

    return positions, colors, scales, opacity, rotations

def quaternion_to_rotation_matrix(q):
    """
    Converts a quaternion [w, x, y, z] into a 3x3 rotation matrix.
    """
    w, x, y, z = q
    return np.array([
        [1 - 2*y**2 - 2*z**2, 2*x*y - 2*w*z, 2*x*z + 2*w*y],
        [2*x*y + 2*w*z, 1 - 2*x**2 - 2*z**2, 2*y*z - 2*w*x],
        [2*x*z - 2*w*y, 2*y*z + 2*w*x, 1 - 2*x**2 - 2*y**2]
    ])

def create_splats(positions, colors, scales, opacity, rotations, sphere_resolution=10):
    """
    Creates Gaussian splats by rendering each point as an ellipsoid.
    """
    splats = []
    base_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1.0, resolution=sphere_resolution)
    
    for i in range(len(positions)):
        # Create a copy of base sphere (ellipsoid)
        ellipsoid = copy.deepcopy(base_sphere)
        
        # Scale it to match splat size
        scale_matrix = np.diag(scales[i])  # Scale along x, y, z
        rotation_matrix = quaternion_to_rotation_matrix(rotations[i])  # Get rotation matrix
        transform_matrix = np.eye(4)
        transform_matrix[:3, :3] = rotation_matrix @ scale_matrix  # Apply rotation and scaling
        transform_matrix[:3, 3] = positions[i]  # Apply translation
        
        # Transform ellipsoid
        ellipsoid.transform(transform_matrix)
        
        # Assign color (opacity can't be applied directly in Open3D)
        ellipsoid.paint_uniform_color([colors[i][0], colors[i][1], colors[i][2]])
        
        splats.append(ellipsoid)

    return splats

def visualize_gaussian_splats(ply_file, num_vertices=None):
    """
    Reads the PLY file, subsamples it (if specified), and visualizes it as Gaussian splats.
    """
    positions, colors, scales, opacity, rotations = read_splat_ply(ply_file, num_vertices)
    splats = create_splats(positions, colors, scales, opacity, rotations)
    o3d.visualization.draw_geometries(splats)

# Example usage:
ply_file_path = "corner2_short.ply"
visualize_gaussian_splats(ply_file_path, num_vertices=2000)  # Subsamples to 2000 points


In [8]:
import open3d as o3d
import numpy as np

# Check what was loaded
print("Points:", np.asarray(pcd.points).shape)
print("Colors:", np.asarray(pcd.colors).shape)  # If color exists
print("Normals:", np.asarray(pcd.normals).shape)


Points: (371067, 3)
Colors: (0, 3)
Normals: (371067, 3)


In [13]:
from plyfile import PlyData

# Read the binary PLY file
ply_file = "C:/Users/nguye/Downloads/images/test.ply"
ply_data = PlyData.read(ply_file)

# Extract vertex data
vertex_data = ply_data['vertex']

# Extract position (x, y, z)
points = np.vstack([vertex_data['x'], vertex_data['y'], vertex_data['z']]).T

# Extract color (f_dc_0, f_dc_1, f_dc_2) and normalize to [0,1]
# colors = np.vstack([vertex_data['f_dc_0'], vertex_data['f_dc_1'], vertex_data['f_dc_2']]).T
# colors = np.clip(colors, 0, 1)  # Ensure values are within range

# Create Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
# pcd.colors = o3d.utility.Vector3dVector(colors)

# Visualize with color
o3d.visualization.draw_geometries([pcd])


In [6]:
# Load the PLY file
corner1_file = "corner2_short.ply"
corner1 = o3d.io.read_point_cloud(corner1_file)

# Visualize the point cloud
# o3d.visualization.draw_geometries([pcd])

print("Points:", np.asarray(corner1.points).shape)

Points: (371067, 3)
